In [1]:
import h5py
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
import sys
import time
import pandas as pd
import cv2
import os
import imageio
import matplotlib as mpl
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import shutil
import random

from deepcell.utils.tracking_utils import load_trks

/opt/conda/lib/python3.7/site-packages/tensorflow-2.5.0-py3.7-linux-x86_64.egg/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [2]:
# get well by well, and store
d4 = [180, 710, 160, 690]
d3 = [180, 710, 880, 1410]
d2 = [180, 710, 1600, 2130]
d1 = [180, 710, 2320, 2850]

c4 = [895, 1425, 160, 690]
c3 = [895, 1425, 880, 1410]
c2 = [895, 1425, 1600, 2130]
c1 = [895, 1425, 2320, 2850]

b4 = [1620, 2150, 160, 690]
b3 = [1620, 2150, 880, 1410]
b2 = [1620, 2150, 1600, 2130]
b1 = [1620, 2150, 2320, 2850]

a4 = [2340, 2870, 160, 690]
a3 = [2340, 2870, 880, 1410]
a2 = [2340, 2870, 1600, 2130]
a1 = [2340, 2870, 2320, 2850]

In [4]:
def get_raw_well(well, video_path="../22956814/22956814.mp4", well_name='A4', filename=False, frame_start=0, frame_end=9001, down_sampling=1):    
    print('--------', well_name, '--------')
    frames = []
    
    ymin = well[0]
    ymax = well[1]
    xmin = well[2]
    xmax = well[3]

    #path = "../22956814/22956814.mp4"
    cap = cv2.VideoCapture(video_path)
    ret = True
    while ret:
        ret, img = cap.read() # read one frame from the 'capture' object; img is (H, W, C)
        if ret:
            frames.append(img[ymin:ymax, xmin:xmax, :])
    
    stacked = np.stack(frames, axis=0)
    stacked = stacked.astype(dtype='float32')
    stacked_mean = np.mean(stacked, axis=3)[:,:,:,None]
    
    if frame_start != 0 or frame_end != 9001 or down_sampling != 1:
        stacked_mean = stacked_mean[frame_start:(frame_end+1):down_sampling, :, :, :]
    
    if filename:
        output_fn = filename
    else:
        output_fn = 'well_A/raw_%s.npy' % well_name
        
    np.save(output_fn, stacked_mean)
    
    #return stacked_mean

In [6]:
get_raw_well(a1, well_name='A1', filename='../22956814/raw_wells/A1.npy')

-------- A1 --------


In [7]:
# Check available memory

import psutil
psutil.virtual_memory()  # looks good!

svmem(total=439595151360, available=432118231040, percent=1.7, used=4118065152, free=423673790464, active=3619418112, inactive=11128471552, buffers=59043840, cached=11744251904, shared=9056256, slab=596033536)

In [14]:
# Files to be removed
fn_remove = []

In [15]:
import gc
import os

raw_video = np.load('../22956814/raw_wells/A1.npy')

mask_frame_info_fn = "../22956814/%s_not_empty.txt" % 'A1'
with open(mask_frame_info_fn, 'r') as f:
    start_end_samplings = f.read().splitlines()  # Cool!! 
    for i, each in enumerate(start_end_samplings):
        frame_start, frame_end, down_sampling = each.split(",")
        fn = "../22956814/%s_raw_%d.npy" % ('A1', (i+1))
        
        frame_start = int(frame_start)
        frame_end = int(frame_end)
        down_sampling = int(down_sampling)
        
        raw_array = raw_video[frame_start:frame_end+1:down_sampling, :, :, :]
        print(raw_array.shape)
        
        # IMPORTANT!!
        if raw_array.shape[0] != 100:
            print("HAVE TO REMOVE: ", fn, " and its masked npy!")
            fn_remove.append(fn)
        
        np.save(fn, raw_array)
        
        # Clear the raw_array memory
        del raw_array
        gc.collect()

# At the end, clear raw_video memory
del raw_video
gc.collect()

(100, 530, 530, 1)
(52, 530, 530, 1)
HAVE TO REMOVE:  ../22956814/A1_raw_2.npy  and its masked npy!


0

In [16]:
import psutil
psutil.virtual_memory()  # looks good!

svmem(total=439595151360, available=432070303744, percent=1.7, used=4161957888, free=423449096192, active=13742714880, inactive=1184972800, buffers=60174336, cached=11923922944, shared=9056256, slab=623755264)

In [17]:
# Other wells
get_raw_well(a2, well_name='A2', filename='../22956814/raw_wells/A2.npy')
get_raw_well(a3, well_name='A3', filename='../22956814/raw_wells/A3.npy')
get_raw_well(a4, well_name='A4', filename='../22956814/raw_wells/A4.npy')

-------- A2 --------
-------- A3 --------
-------- A4 --------


In [19]:
# Make a function from above

import gc
import os

def make_raw_videos(raw_video_path, mask_frame_info_fn, well):
    #raw_video = np.load('../22956814/raw_wells/A1.npy')
    raw_video = np.load(raw_video_path)

    #mask_frame_info_fn = "../22956814/%s_not_empty.txt" % 'A1'
    
    with open(mask_frame_info_fn, 'r') as f:
        start_end_samplings = f.read().splitlines()  # Cool!! 
        for i, each in enumerate(start_end_samplings):
            frame_start, frame_end, down_sampling = each.split(",")
            fn = "../22956814/%s_raw_%d.npy" % (well, (i+1))

            frame_start = int(frame_start)
            frame_end = int(frame_end)
            down_sampling = int(down_sampling)

            raw_array = raw_video[frame_start:frame_end+1:down_sampling, :, :, :]
            print(raw_array.shape)

            # IMPORTANT!!
            if raw_array.shape[0] != 100:
                print("HAVE TO REMOVE: ", fn, " and its masked npy!")
                fn_remove.append(fn)

            np.save(fn, raw_array)

            # Clear the raw_array memory
            del raw_array
            gc.collect()

    # At the end, clear raw_video memory
    del raw_video
    gc.collect()